In [ ]:
%matplotlib inline
store_matrices_given_param_delta_t = []
store_unique_objects_delta_t = []
store_masked_array_delta_t = []
for param in ordered_list_of_parameters_sublist:
    FOCAL_BAT, INCLUDE_DIRECT_SOUNDS, AZIMUTH  = param
    track_unique_objects = []

    heard_sounds_array = dictionary_to_store_heard_sounds[param]
    
    nan_matrix_delta_t_self_echoes = (
        np.zeros(
            shape=(len(heard_sounds_array), NUM_BATS - 1 + 4)
        )
        * np.nan
    )
    nan_matrix_delta_t_direct = (
        np.zeros(
            shape=(len(heard_sounds_array), NUM_BATS - 1 + 4)
        )
        * np.nan
    )
    nan_matrix_delta_t_non_self_echoes = (
        np.zeros(
            shape=(len(heard_sounds_array), NUM_BATS - 1 + 4)
        )
        * np.nan
    )
    
    for i, frame_heard_sounds in enumerate(heard_sounds_array):
        
        for sound_object in frame_heard_sounds:

            time = (
                np.array(sound_object["occurance_times"])[0]
                - sound_object["bat_last_call_time"]
                + i * 0.04
            )
            theta = sound_object["bat_direction"].angle_between(
                sound_object["incident_direction"]
            )

            if sound_object["emitter_id"] == FOCAL_BAT:
                reflected_from = sound_object["reflected_from"]
                if reflected_from not in track_unique_objects:
                    track_unique_objects.append(reflected_from)
                index = track_unique_objects.index(reflected_from)
                
                nan_matrix_delta_t_self_echoes[i, index] = theta
                
            elif sound_object["emitter_id"] != FOCAL_BAT and sound_object["type"] == "echo":
                reflected_from = sound_object["reflected_from"]
                if reflected_from not in track_unique_objects:
                    track_unique_objects.append(reflected_from)
                index = track_unique_objects.index(reflected_from)
                # print(track_unique_objects)
                nan_matrix_delta_t_non_self_echoes[i, index] = theta
            
            elif sound_object["emitter_id"] != FOCAL_BAT and sound_object["type"] == "direct":
                emitter = f"bat_{sound_object["emitter_id"]}"
                if emitter not in track_unique_objects:
                    track_unique_objects.append(emitter)
                index = track_unique_objects.index(emitter)
                
                nan_matrix_delta_t_direct[i, index] = theta
        
    nan_matrix_delta_t_direct, unique_direct, masked_array_direct= reformat_matrices(nan_matrix_delta_t_direct, track_unique_objects)
    nan_matrix_delta_t_self_echoes, unique_self_echoes_from, masked_array_self_echoes = reformat_matrices(nan_matrix_delta_t_self_echoes, track_unique_objects)  
    nan_matrix_delta_t_non_self_echoes, unique_non_self_echoes_from, masked_array_non_self_echoes = reformat_matrices(nan_matrix_delta_t_non_self_echoes, track_unique_objects)
    
    store_matrices_given_param_delta_t.append([nan_matrix_delta_t_self_echoes, nan_matrix_delta_t_direct, nan_matrix_delta_t_non_self_echoes])
    store_unique_objects_delta_t.append(unique_self_echoes_from)
    store_masked_array_delta_t.append([masked_array_self_echoes, masked_array_direct, masked_array_non_self_echoes])
    

In [ ]:
for i, param in enumerate(ordered_list_of_parameters_sublist):
    FOCAL_BAT, INCLUDE_DIRECT_SOUNDS, AZIMUTH  = param
    
    list_of_matrices = store_matrices_given_param_delta_t[i]
    track_unique_objects = store_unique_objects_delta_t[i]
    masked_arrays_given_param = store_masked_array_delta_t[i]
    
    cmap_masked = mpl.cm.jet
    cmap_masked.set_bad('black',1.)
    
    
    plt.figure(figsize=(40, 10))
    plt.subplot(3,1,1)
    plt.imshow(list_of_matrices[0], cmap=mpl.colormaps["hsv"])
    plt.colorbar(
        location="top",
        label="angle of incidence",
    )
    plt.ylabel("self echo reflected from")
    plt.xlabel("interpulse interval")
    plt.yticks(
        labels=track_unique_objects, ticks=np.arange(0, nan_matrix.shape[0])
    )
    plt.imshow(masked_arrays_given_param[i], interpolation='nearest', cmap=cmap_masked)
    plt.clim(0.005,0.035)
    
    plt.subplot(3,1,2)
    plt.imshow(list_of_matrices[1], cmap=mpl.colormaps["hsv"])
    plt.colorbar(
        location="top",
        label="angle of incidence",
    )
    plt.ylabel("self echo reflected from")
    plt.xlabel("interpulse interval")
    plt.yticks(
        labels=track_unique_objects, ticks=np.arange(0, nan_matrix.shape[0])
    )
    plt.clim(0.005,0.035)
    
    plt.subplot(3,1,3)
    plt.imshow(list_of_matrices[2], cmap=mpl.colormaps["hsv"])
    plt.colorbar(
        location="top",
        label="angle of incidence",
    )
    plt.ylabel("self echo reflected from")
    plt.xlabel("interpulse interval")
    plt.yticks(
        labels=track_unique_objects, ticks=np.arange(0, nan_matrix.shape[0])
    )
    plt.clim(0.005,0.035)
    
    plt.suptitle(f"{NUM_BATS} bats, parameters focal_bat={FOCAL_BAT}, azimuth_threshold={np.round(AZIMUTH,2)}, include_direct_sounds={INCLUDE_DIRECT_SOUNDS}")
    plt.show()
    plt.clf()